# CHECK IF QAT SUPPORTS  UpsizeBilinear2D

# Install TF optimiazation if NOT INSTALLED!!!!!

In [ ]:
# !rm -rf model-optimization
# !git clone https://github.com/tensorflow/model-optimization.git
# %cd model-optimization
# %pwd
# !python setup.py build
# !python setup.py install

In [ ]:
!pip install tensorflow-model-optimization

In [ ]:
!pip list | grep tensorflow

In [ ]:
import tensorflow_model_optimization as tfmot

In [ ]:
!pip check

In [ ]:

import os
import numpy as np
import glob
import datetime
from random import shuffle


import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
import tensorflow_model_optimization as tfmot # <------------------------------------------- this is the key



In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
tf.__version__

In [ ]:
tf.keras.backend.image_data_format()

In [ ]:
print(tfmot)

In [ ]:
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

NGPU = len(get_available_gpus())
NGPU

# Hyper Params

In [ ]:
BATCH_SIZE = 256
EPOCHS = 1
LEARNING_RATE = 1e-4
INPUT_MAX = 1.0 # 255.0

# for plot

In [ ]:
import matplotlib.pyplot as plt
 
def plot_gt_and_predictions(model, test_images, is_plot=True):
    predictions = model.predict(test_images[:16,...])
    predictions = np.clip(predictions, 0, INPUT_MAX)
    
    if is_plot:
        fig = plt.figure(figsize=(16, 16))
        for i in range(predictions.shape[0]):
            plt.subplot(4, 8, 2*i + 1)
            plt.imshow(test_images[i, :, :, :] )
            plt.axis('off')

            plt.subplot(4, 8, 2*i + 2)
            plt.imshow(predictions[i, :, :, :])
            plt.axis('off')



        plt.show()
    
    return predictions


def plot_two_predictions(images1, images2):
    images1 = np.clip(images1, 0, INPUT_MAX)
    images2 = np.clip(images2, 0, INPUT_MAX)
    
     
    fig = plt.figure(figsize=(16, 16))
    for i in range(images1.shape[0]):
        plt.subplot(4, 8, 2*i + 1)
        plt.imshow(images1[i, :, :, :], vmin=0, vmax=INPUT_MAX )
        plt.axis('off')
        plt.subplot(4, 8, 2*i + 2)
        plt.imshow(images2[i, :, :, :], vmin=0, vmax=INPUT_MAX)
        plt.axis('off')


 
    plt.show()
    
    return None







def generate_and_save_images(model, epoch, test_images):
    predictions = model.predict(test_images[:16,...])
    predictions = np.clip(predictions, 0, INPUT_MAX)
    
     
    fig = plt.figure(figsize=(16, 16))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 8, 2*i + 1)
        plt.imshow(test_images[i, :, :, :] , vmin=0, vmax=INPUT_MAX)
        plt.axis('off')
        
        plt.subplot(4, 8, 2*i + 2)
        plt.imshow(predictions[i, :, :, :], vmin=0, vmax=INPUT_MAX)
        plt.axis('off')


 
    plt.show()
    
    return predictions

In [ ]:
def plot_images(images):
    images = images[:16,...]
    images = np.clip(images, 0, INPUT_MAX)
    
     
    fig = plt.figure(figsize=(8, 8))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i + 1)
#         im = plt.imshow(images[i, :, :, :].astype(np.uint8))
        im = plt.imshow(images[i, :, :, :])
        plt.axis('off')
  
    plt.show()
    
    
    
    
def plot_multi_images(images): # images: list
    num_sets = len(images)
    print('--------------num_Sets', num_sets)
    
    images = np.clip(images, 0, INPUT_MAX)
    images = images*255
    images = images.astype(np.uint8)
     
    fig, axes = plt.subplots(nrows=8, ncols=2*num_sets, figsize=(16, 16))
    cnt=0
    for ax in axes.flat:
        
        im = ax.imshow(images[cnt%num_sets][cnt//num_sets, :, :, :], vmin=0, vmax=255 )
        
        ax.set_xticks([])
        ax.set_yticks([]) 
        cnt+=1
#     fig.colorbar(im, ax=axes.ravel().tolist())

    plt.show()
    
    
    
    
def plot_images_with_color_bar(images):
    images = images[:16,...]
    images = np.clip(images, 0, INPUT_MAX)
    images = images*255
    images = images.astype(np.uint8)
    
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(8, 8))
    cnt=0
    for ax in axes.flat:
        im = ax.imshow(images[cnt, :, :, :], vmin=0, vmax=255, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([]) 
        
        cnt+=1
        
    fig.colorbar(im, ax=axes.ravel().tolist())

    plt.show()
    
    

# Define model 

In [ ]:

def get_model(name='bwae'):

    ch1 = 64
    ch2 = 128
    ch3 = 256
    ch4 = 256


    ch1 = 16
    ch2 = 32
    ch3 = 64
    ch4 = 128

    connections = [True, True, True]
    channels = [ch3, ch2, ch1]



    iinput = tf.keras.layers.Input((32, 32, 3), name="input")



    x1 = tf.keras.layers.Conv2D(ch1, 3, strides=1, activation="relu", padding="same", name="enc1" )(iinput)
    x2 = tf.keras.layers.Conv2D(ch2, 3, strides=2, activation="relu", padding="same", name="enc2" )(x1)
    x3 = tf.keras.layers.Conv2D(ch3, 3, strides=2, activation="relu", padding="same", name="enc3" )(x2)
    x4 = tf.keras.layers.Conv2D(ch4, 3, strides=2, activation="relu", padding="same", name="enc4" )(x3)
    x = x4

    lrsc = [x3, x2, x1]
    for i in range(len(connections)):
        namet = 'dec%d' %i
        namec = 'dec%d_conv' % i
        name_a = 'add%d' % i

        activationt = None if connections[i] else "relu"

        x = tf.keras.layers.Conv2DTranspose(channels[i], 3, strides=2, activation=activationt, padding="same", name=namet)(x)
 


        if connections[i]:
            x = tf.keras.layers.Add(name=name_a)([x, lrsc[i] ])
        x = tf.keras.layers.Conv2D(ch3, 3, strides=1, padding="same", name=namec, activation="relu")(x)




    x = tf.keras.layers.Conv2D(3, 3, strides=1, activation="relu", padding="same", name="last_conv")(x)


    model = tf.keras.models.Model(iinput, x, name=name)
    
    return model






In [ ]:
model = get_model()


model.save('bwae.h5')
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, dpi=64)


# Load CIFAR10

In [ ]:
cifar10 = keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images = (train_images / 255.0) * INPUT_MAX
test_images = (test_images / 255.0) * INPUT_MAX

print(type(train_images[0,0,0,0]))

train_images = train_images.astype(np.float32)
test_images = test_images.astype(np.float32)
print(type(train_images[0,0,0,0]))


In [ ]:
print(np.amin(test_images),np.amax(test_images))

In [ ]:
plt.subplot(141)
plt.imshow(train_images[0], interpolation="bicubic") 
plt.grid(False)
plt.subplot(142)
plt.imshow(train_images[4], interpolation="bicubic")
plt.grid(False)
plt.subplot(143)
plt.imshow(train_images[8], interpolation="bicubic")
plt.grid(False)
plt.subplot(144)
plt.imshow(train_images[12], interpolation="bicubic")
plt.grid(False)
plt.show()

# before traning

In [ ]:
before = plot_gt_and_predictions(model, test_images[:16,...])

# Train - float

In [ ]:
optimizer_f = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, name='Adam_f')
model.compile(optimizer=optimizer_f, loss=["mse"])

history = model.fit(train_images, 
                    train_images,  
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    validation_data=(test_images, test_images))

model.save('bwae.h5')



# Plot Float Model

In [ ]:
predictions_float = plot_gt_and_predictions(model, test_images[:16,...])

# Make Quantized Model

In [ ]:
# import tensorflow_model_optimization as tfmot # <---------------------------------------- remember?

base_model = get_model('bwae_quant')

base_model.load_weights('bwae.h5')

quant_aware_model = tfmot.quantization.keras.quantize_model(base_model)
quant_aware_model.save('bwae_qat_before_train.h5')
quant_aware_model.summary()

tf.keras.utils.plot_model(quant_aware_model, show_shapes=True, dpi=64)





# Train Q Awarre model

In [ ]:
optimizer_q = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, name='Adam_f')
quant_aware_model.compile(optimizer=optimizer_q, loss=["mse"])

history = quant_aware_model.fit(train_images, 
                            train_images,  
                            batch_size=BATCH_SIZE, 
                            epochs=EPOCHS, 
                            validation_data=(test_images, test_images))

quant_aware_model.save('bwae_qat_after_train.h5')

# Plot -QAT

In [ ]:
predictions_qat = plot_gt_and_predictions(quant_aware_model, test_images[:16,...])
# predictions_qat = generate_and_save_images(q_aware_model, 0, train_images[:16,...])
print(predictions_qat.shape)
print(np.amin(predictions_qat), np.amax(predictions_qat))

# Tflite Convert - float

In [ ]:
input_name = model.input_names[0]
index = model.input_names.index(input_name)
model.inputs[index].set_shape([1, 32, 32, 3])

converter = tf.lite.TFLiteConverter.from_keras_model(model) 
tflite_float_model = converter.convert()
fname = "bwae.tflite"
open(fname, 'wb').write(tflite_float_model)

# Tflite Convert - qat

In [ ]:
input_name = quant_aware_model.input_names[0]
index = quant_aware_model.input_names.index(input_name)
quant_aware_model.inputs[index].set_shape([1, 32, 32, 3])
 

converter = tf.lite.TFLiteConverter.from_keras_model(quant_aware_model) 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_qat_model = converter.convert()
fname = "bwae_qat.tflite"
open(fname, 'wb').write(tflite_qat_model)

# Diff: Float vs QAT

In [ ]:
diff = predictions_float - predictions_qat
abs_diff = np.abs(diff)
abs_diff = np.sum(abs_diff, axis=-1)
abs_diff = abs_diff[..., np.newaxis]

plot_images_with_color_bar(abs_diff)
plot_two_predictions(predictions_float, predictions_qat)
# plot_images(abs_diff)


In [ ]:
plot_multi_images([test_images[:16,...], predictions_float, predictions_qat])

# REtrain

In [ ]:
base_model = get_model('bwae_quant_retrain')
# base_model.load_weights('bwae_qat_after_train.h5') # <---------- NOT here


qmodel = tfmot.quantization.keras.quantize_model(base_model)
qmodel.load_weights('bwae_qat_after_train.h5') # <---------- here
qmodel.save('bwae_qat_before_retrain.h5')
qmodel.summary()

tf.keras.utils.plot_model(qmodel, show_shapes=True, dpi=64)

In [ ]:
    
optimizer_r = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, name='Adam_f')
qmodel.compile(optimizer=optimizer_r, loss=["mse"])

history = qmodel.fit(train_images, 
                     train_images,  
                     batch_size=BATCH_SIZE, 
                     epochs=EPOCHS, 
                     validation_data=(test_images, test_images))

qmodel.save('bwae_qat2.h5', include_optimizer=False)

# Save REtrained weights

In [ ]:
# input_name = model.input_names[0]
# index = model.input_names.index(input_name)
# qmodel.inputs[index].set_shape([1, 32, 32, 3])


# converter = tf.lite.TFLiteConverter.from_keras_model(qmodel) 
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_qmodel = converter.convert()
# fname = "bwae_qat2.tflite"
# open(fname, 'wb').write(tflite_qmodel)

# Prediction of REtrained QAT


In [ ]:
predictions_qat2 = plot_gt_and_predictions(qmodel, test_images[:16,...], False)

In [ ]:
plot_multi_images([test_images[:16,...], predictions_float, predictions_qat, predictions_qat2])

# Save REtrained weights


In [ ]:
input_name = model.input_names[0]
index = model.input_names.index(input_name)
qmodel.inputs[index].set_shape([1, 32, 32, 3])


converter = tf.lite.TFLiteConverter.from_keras_model(qmodel) 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_qmodel = converter.convert()
fname = "bwae_qat2.tflite"
open(fname, 'wb').write(tflite_qmodel)